<a href="https://colab.research.google.com/github/chunam76/LLM/blob/main/%5B%EC%8B%A4%EC%8A%B5%5D_3_OpenAI_API_%EA%B8%B0%EB%B0%98_%EC%9B%B9_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91_%EB%89%B4%EC%8A%A4_%EB%B6%84%EC%84%9D_%EB%B4%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [실습] OpenAI API를 이용하여 뉴스 기사 정리 봇 만들기

OpenAI Chat API를 이용해 뉴스 검색 결과를 정리하는 프로그램을 만들어 보겠습니다.


크롤링 전용 라이브러리가 필요하지만, 이번 실습에서는 네이버의 뉴스 검색 API를 사용하겠습니다.

In [ ]:
!pip install openai tiktoken --upgrade

In [ ]:
# os의 환경 변수에 API 키 복사 붙여넣기
import openai
import os

# OPENAI API KEY 설정
os.environ['OPENAI_API_KEY']=""

client = openai.OpenAI()


assert len(os.environ['OPENAI_API_KEY']) > 0, "OPENAI_API_KEY가 환경 변수에 설정되어 있지 않습니다. API 키를 설정해주세요."

# API 키가 설정되어 있다면, 이 지점 이후의 코드가 실행됩니다.
print("OPENAI_API_KEY가 정상적으로 설정되어 있습니다.")

## 1. LLM으로 웹 크롤링 코드 구현 프롬프트 만들기

네이버 뉴스 API를 이용해 크롤링을 수행하는 파이썬 코드를 작성하겠습니다.   
검색어가 `query` 인수로 주어지면, 관련도순으로 검색된 뉴스 기사 30개의 정보를 {"title", "URL", "desc"} 의 dictionary 형식으로 파싱하고,   
이를 리스트로 만들어 json으로 return하는 함수 get_html(query) 를 만들고 싶습니다.   <br>
   
위 요청을 그대로 전달하면, 함수를 만들 수 있을 것 같습니다.


**네이버 뉴스 API를 사용한다는 내용을 명시하면 정확도가 올라갑니다.**

In [ ]:
prompt_instuction = '''
네이버 뉴스 API를 이용해 크롤링을 수행하는 파이썬 코드를 작성해 주세요.
검색어가 `query` 인수로 주어지면, 관련도순으로 검색된 뉴스 기사 30개의 정보를 {"title", "URL", "desc"} 의 dictionary 형식으로 파싱하고,
이를 리스트로 만들어 json으로 return하는 함수 get_html(query) 를 만들면 됩니다.
---
'''

prompt = '''

# 검색어
query = '삼성SDS 생성형 AI'


# 검색 결과를 return하는 함수
def get_html(query):


'''



In [ ]:
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [{'role':'system', 'content':'당신은 파이썬 코딩의 전문가입니다.'},
          {'role':'user', 'content':prompt_instuction + prompt}],

    temperature =  0.1,
    max_tokens = 1000

)
print(response.choices[0].message.content)

In [ ]:
# VS CODE에서 위 결과가 truncated 되는 경우 scrollable element를 클릭하여 풀 코드 확인

아래의 API 키를 복사하여 넣으면 됩니다.

In [ ]:
# 네이버 API 키: 개발자 센터에서 발급 가능

headers = {
    'X-Naver-Client-Id': 'Ko6yIqbV2TOHq9rPH8tu',
    'X-Naver-Client-Secret': 'BvqX8mNtHu'
}

아래에 전체 코드를 넣고 실행합니다.

아래는 예시 코드입니다.

In [ ]:
# # 전체 코드를 선택 후 Ctrl+/ 로 일괄 주석해제
# import requests
# import json

# def get_html(query):
#     # 네이버 API URL
#     url = "https://openapi.naver.com/v1/search/news.json"

#     # API 요청 헤더
#     headers = {
#     'X-Naver-Client-Id': 'Ko6yIqbV2TOHq9rPH8tu',
#     'X-Naver-Client-Secret': 'BvqX8mNtHu'

#     }

#     # API 요청 파라미터
#     params = {
#         "query": query,
#         "display": 30,  # 검색 결과 개수
#         "sort": "sim"   # 관련도순 정렬
#     }

#     # API 요청
#     response = requests.get(url, headers=headers, params=params)

#     # 응답이 성공적일 경우
#     if response.status_code == 200:
#         data = response.json()
#         articles = data.get('items', [])

#         # 결과를 저장할 리스트
#         result = []

#         # 기사 정보 파싱
#         for article in articles:
#             result.append({
#                 "title": article['title'],
#                 "URL": article['link'],
#                 "desc": article['description']
#             })

#         # JSON 형식으로 반환
#         return json.dumps(result, ensure_ascii=False)

#     else:
#         return json.dumps({"error": "Failed to fetch data"}, ensure_ascii=False)

# # 검색어
# query = '삼성SDS 생성형 AI'

# # 함수 호출
# result = get_html(query)
# print(result)

## 2. OpenAI chat을 이용하여 요약하기

키워드에 대한 뉴스 검색 결과를 요약합니다.   
위 결과인 news_json 데이터를 입력으로 넣은 뒤, 해당 결과에 대한 요약을 요청합니다.

In [ ]:
def news_bot(messages):

    result = client.chat.completions.create(
        model="gpt-4o-mini",
        messages = messages,
        temperature = 0.1,
        max_tokens= 2048,
    )

    return result.choices[0].message.content

챗봇에 전달할 메시지는 두 개로 나눠집니다.   
위에서 얻은 get_html(query)와, 사용자가 제시할 명령입니다.   

In [ ]:
query = '삼성 라이온즈'

news_json = get_html(query)

result = news_bot([
    {
        "role": "system",
        "content": query + """에 대한 뉴스 검색 결과가 주어집니다. """+ query+"""에 대한 뉴스를
요약하세요.
---
"""
    },
    {
        "role": "user",
        "content": get_html(query)
    }
])

result

In [ ]:
# 토큰 수 체크

import tiktoken
tokenizer  = tiktoken.encoding_for_model('gpt-4o-mini')

print("문자 수:", len(news_json))
print("토큰 수:", len(tokenizer.encode(news_json)))
news_json


다양한 명령을 넣어서 형식과 스타일을 바꿀 수 있습니다.

In [ ]:
query = '삼성 라이온즈'

news_json  = get_html(query)

result = news_bot([
    {
        "role": "user",
        "content": news_json
    },
    {
        "role": "user",
        "content": f"""
---
위 전체 내용을 종합하여 {query}의 동향과 미래에 대해 논쟁하는 두 사람의 대화 내용을 만들어줘. 한 명은 전문적인 말투, 한 명은 어리숙한 말투를 사용하고, 형식은 아래와 같아.
---
A:(A의 대화)
B:(B의 대화)
        """
    }
])

print(result)

## 참고) 스트리밍
ChatGPT처럼 글자가 한 글자씩 순서대로 쭉 출력되도록 하면 어떨까요?   
이와 같은 작업을 스트리밍(Streaming)이라고 하며, 옵션을 추가하여 구현할 수 있습니다.

In [ ]:
def news_bot_stream(messages):

    result = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages,
        temperature = 0.1,
        max_tokens = 2048,
        stream=True # 스트리밍
    )

    return result

In [ ]:
query = '삼성SDS'
news_json = get_html(query)

result = news_bot_stream([
    {
        "role": "user",
        "content": news_json
    },
    {
        "role": "user",
        "content": """
위 전체 내용을 종합하여 친구에게 전달하는 편지를 작성해줘. 친절하고 상냥한 말투를 사용하고, 형식은 아래와 같아.
---
'제목':
'본문':
        """
    }
])

for chunk in result:
    print(chunk.choices[0].delta.content, end='')

이와 같은 스트리밍은 LLM 어플리케이션의 마지막 출력에서 사용하면    
사용자 경험을 향상시킬 수 있습니다.

<br><br><br>
## Tool(과거 Function Call) : 커스텀 함수 사용하기
만약 query를 명시적으로 지정하지 않고, 사용자의 질문에 따라 관련 query를 추출하고,  
이를 통해 검색하게 할 수도 있을까요?    

`Tool Calling` 를 추가하면 이를 수행할 수 있습니다.    

Web_Search라는 이름의 툴을 만들고, 이를 `get_html`과 연결하겠습니다.

-------


In [ ]:
from pydantic import BaseModel, Field

class Web_Search(BaseModel):
    """질문이 들어오면, 핵심 키워드로 뉴스를 검색하여 그 결과를 저장합니다.
매개변수 query는 핵심 키워드로, 한국어로 작성합니다."""
    query: str= Field(description="검색 키워드")

tools = [openai.pydantic_function_tool(Web_Search)]
tools

Tool 정보가 포함된 gpt는 함수의 구조와 설명을 이용하여 어떤 함수를 써야 하는지를 판단합니다.

직접적으로 실행하는 것은 아니지만, 결과를 활용하면 해당 함수를 바로 사용할 수 있습니다.

In [ ]:
def query_bot(messages):

    # 사용할 툴 펑션의 목록과 설명을 리스트로 전달
    # LLM이 스스로 description, name, parameter의 값을 통해 판단
    class Web_Search(BaseModel):
        """질문이 들어오면, 핵심 키워드로 뉴스를 검색하여 그 결과를 저장합니다.
매개변수 query는 핵심 키워드로, 한국어로 작성합니다."""
        query: str= Field(description="검색 키워드")

    tools = [openai.pydantic_function_tool(Web_Search)]



    result = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages,
        tools = tools,
        tool_choice = 'auto',
        # 'none'이면 툴 사용하지 않음
        # 'required'면 무조건 툴 사용
        temperature= 0.1,
        max_tokens= 1024,
    )

    return result.choices[0].message

In [ ]:
result = query_bot([
    {
        "role": "user",
        "content": """안녕하세요! 오늘 날씨가 좋네요."""
    }
])

result

In [ ]:
result = query_bot([
    {
        "role": "user",
        "content": """
요즘 새로 개봉한 영화는 무엇이 있나요?
"""
    }
])

result

LLM이 스스로 판단하여, 실행할 함수의 정보를 `tool_calls`에 돌려줍니다.

In [ ]:
result.tool_calls[0].function.name, result.tool_calls[0].function.arguments

함수명과 함수에 전달할 매개변수값이 주어지면, 아래와 같이 함수를 불러올 수 있습니다.

In [ ]:
# 문자열을 Dict로 바꾸기

example = '{"query":"LLM"}'

example_dict = json.loads(example)
# json으로 변환

print(example_dict)
print(type(example_dict))



In [ ]:
# 함수이름 문자열을 함수로 바꾸기

available_functions = {'Web_Search': get_html}

available_functions['Web_Search'](query = 'LLM')

In [ ]:
# Dictionary를 함수의 매개변수로 전달하기

def f(a,b):
  return a + b

example2 = {'a': 1, 'b': 2}

f(**example2)


In [ ]:
get_html(**example_dict)

`query_bot`, `get_html`, `news_bot` 을 조합하여 실행해 보겠습니다.

In [ ]:
# example
# "오늘 가장 큰 뉴스는 뭐야?"
# """오픈AI는 요즘 새로운 소식 있어?"""

prompt = """넷플릭스 신작 추천해줘."""
print('Prompt:',prompt)


# 출력 결과를 함수와 연결하기 위한 dictionary
available_functions = {'Web_Search':get_html}


# query_bot은 프롬프트를 받아 get_html을 수행할지 결정합니다.
# 단순 대화라면 tool call을 생성하지 않습니다.
result = query_bot([
    {
        "role": "user",
        "content": prompt
    }
])

print('---')
print('Query_Bot: Call ', end='')



if result.tool_calls: # tool_calls가 존재하면:

    name, arguments = result.tool_calls[0].function.name, result.tool_calls[0].function.arguments
    print(name,arguments)
    # 이름과 인수 출력


    arguments = json.loads(arguments)
    # Str To Dict

    news_json = available_functions[name](**arguments)
    # 함수 선택, 매개변수 전달

    print('---')
    print('News_Bot:',end='')



    # news_bot_stream으로 요약 수행
    result = news_bot_stream([
        {
            "role": "system",
            "content": """뉴스 검색 결과와 질문이 주어집니다.
검색 결과를 바탕으로 질문에 답변하세요.
문장마다 줄바꿈을 사용하고, 만약 링크가 필요하다면 실제 링크를 추가하세요.
친절하고 상냥한 말투를 사용하세요."""
        },
        {
            "role": "user",
            "content": f"""
뉴스 검색 결과 : {news_json}
질문: {prompt}
"""

        }
    ])
    for chunk in result:
       print(chunk.choices[0].delta.content, end='')

else:
    print('Nothing')
    print(result.content)

## 3. 보안 프롬프트 추가하기

위에서 만든 news_bot에, 관련없는 질문을 입력하면 어떻게 될까요?

In [ ]:
news_json

In [ ]:
prompt = '오늘 저녁 뭐 먹을까? 타코와 피자 중에서.'

result = news_bot([
        {
            "role": "system",
            "content": """뉴스 검색 결과와 질문이 주어집니다.
검색 결과를 바탕으로 질문에 답변하세요.
문장마다 줄바꿈을 사용하고, 만약 링크가 필요하다면 실제 링크를 추가하세요.
친절하고 상냥한 말투를 사용하세요."""
        },
        {
            "role": "user",
            "content": f"""
뉴스 검색 결과 : {news_json}
질문: {prompt}
"""

        }
    ])

result

그럭저럭 대답을 잘 하고 있습니다만, query를 통해 검색한 결과와는 관련이 없습니다.   
만약 특정 태스크만을 수행하는 챗봇을 만들었다면, 이와 같은 응답은 **환각(Hallucination)** 으로 간주될 수 있으며,   
이와 같은 행동을 쉽게 허용한다면 예상치 못한 출력을 생성할지도 모릅니다.


환각을 방지하기 위해, 프롬프트를 조금 더 강화시켜 보겠습니다.    
챗봇의 메시지 중 `system`을 이용하면, 챗봇의 행동을 더 명시적으로 전달할 수 있습니다.

In [ ]:
news_json

In [ ]:
prompt = "오늘 저녁 뭐 먹을까? 타코와 피자 중에서."

result = news_bot([
    {"role": "system",
            "content": """[뉴스 검색 결과]와 [질문]이 주어집니다.
검색 결과를 바탕으로 질문에 답변하세요.
문장마다 줄바꿈을 사용하고, 만약 링크가 필요하다면 실제 링크를 추가하세요.
친절하고 상냥한 말투를 사용하세요.
만약 [뉴스 검색 결과]의 내용만으로 답변하기 어려운 경우에는 '답변할 수 없습니다.'만 출력하세요."""},

    {"role": "user",
            "content": f"""
뉴스 검색 결과 : {news_json}
질문: {prompt}
"""}

])

result

이번 실습 시간에 만들어 본 봇의 기능을 더 추가하는 방법에는 무엇이 있을까요?
<br><br>
- 검색 결과에서 최신 기사만 필터링하기
- query 검색 결과에서 특정 단어가 포함된 기사는 제외하기
- 각 기사들의 설명을 보고 해당 단어에 대한 긍정(부정)적인 기사만 사용하여 요약하기    

이와 같은 방법들은 실제로 사용되는 방식이므로, 이를 활용하면 챗봇의 기능을 더 고도화할 수 있습니다.

## 부록) Prompt Injection

적대적 프롬프트(Adversarial Prompt)를 통해 챗봇의 기능을 무력화하고, 정보를 유출시키는 프롬프트도 있습니다.

https://www.promptingguide.ai/kr/risks/adversarial    
https://github.com/0xk1h0/ChatGPT_DAN    
